# Baseline
## Import

In [1]:
# %pip install accelerate
# %pip install -i https://pypi.org/simple/ bitsandbytes
# %pip install transformers[torch] -U

# %pip install datasets
# %pip install langchain
# %pip install langchain_community
# %pip install PyMuPDF
# %pip install sentence-transformers
# %pip install faiss-gpu

In [2]:
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import os
import pickle

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Gemma2ForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
from accelerate import Accelerator

# Train 관련
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from peft import LoraConfig, PeftModel, PeftConfig
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Vector DB

In [3]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

## DB 생성

In [4]:
import os 

In [5]:
train_src_list = os.listdir(os.getcwd()+'/train_source')
print(len(train_src_list))
print(train_src_list)

16
['1-1 2024 주요 재정통계 1권.pdf', '2024 나라살림 예산개요.pdf', '2024년도 성과계획서(총괄편).pdf', '「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf', '「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》.pdf', '고용노동부_내일배움카드(일반).pdf', '고용노동부_조기재취업수당.pdf', '고용노동부_청년일자리창출지원.pdf', '국토교통부_민간임대(융자).pdf', '국토교통부_소규모주택정비사업.pdf', '국토교통부_전세임대(융자).pdf', '보건복지부_노인일자리 및 사회활동지원.pdf', '보건복지부_생계급여.pdf', '월간 나라재정 2023년 12월호.pdf', '재정통계해설.pdf', '중소벤처기업부_창업사업화지원.pdf']


In [6]:
base_directory = './' # Your Base Directory
df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [7]:
test_df

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


## 모델 학습

In [8]:
# X_train = df['Question']
# Y_train = df['Answer']

# # make dataset
# train_data = []
# for x, y in zip(X_train, Y_train):
#     train_data.append({"user":x, "assistant":y})
    
# train_dataset = Dataset.from_list(train_data)

# dataset_dict = DatasetDict({
#     "train": train_dataset
# })

# # save at local
# dataset_dict.save_to_disk('./finance_dataset')

# load dataset from local
dataset_dict = DatasetDict.load_from_disk('./finance_dataset')

dataset_dict['train'][0]

{'user': '2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?',
 'assistant': '2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 21개, 기금 68개로 구성되어 있습니다.'}

In [10]:
def load_model(model_id):
            # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 토크나이저 로드 및 설정
    global tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation='eager'
    )
    
    return model, tokenizer

def generate_prompts_train(example):
    output_texts = []
    for i in range(len(example['user'])):
        messages = [
#             {"role": "system", "content": system_msg},
            {'role': 'user', 'content': "{}".format(example['user'][i])},
            {'role': 'assistant', 'content': "{}".format(example['assistant'][i])}
        ]

        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        chat_message = chat_message.rstrip()
        chat_message = chat_message+"<eos>"
        
        '''
        print("###################################")
        print('###chat_message###\n', chat_message)
        print("###################################")
        '''

        output_texts.append(chat_message)

    return output_texts

def train_model_SFT(model_id, train_dataset, model_save_path):
    model, tokenizer = load_model(model_id)
    
    lora_config = LoraConfig(
        r=8,
        lora_alpha = 16,
        lora_dropout = 0.05,
        target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
    )   

    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        max_seq_length=512,
        args=TrainingArguments(
            output_dir=model_save_path+'_checkpoint_dir',
#             evaluation_strategy = "steps",
#             eval_steps = 5,
#             load_best_model_at_end = True,
            num_train_epochs = 5,
            max_steps=3000,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            optim="paged_adamw_8bit",
            warmup_steps=100,
            learning_rate=2e-4,
            bf16=True, 
            logging_steps=100,
            report_to="wandb",
            run_name=model_save_path,
            push_to_hub=False
        ),
        peft_config=lora_config,
        formatting_func=generate_prompts_train
#         callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    trainer.train()
    
    # model save
    ADAPTER_MODEL = model_save_path+"lora_adapter"

    trainer.model.save_pretrained(ADAPTER_MODEL)

    model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.bfloat16)

    model = model.merge_and_unload()
    model.save_pretrained(model_save_path)
    
    return model, tokenizer

In [11]:
model, tokenizer = train_model_SFT(model_id='google/gemma-2-9b-it', 
                                   train_dataset=dataset_dict['train'], 
                                   model_save_path='finance_gemma2-9b-it_20240822_v2')

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.60s/it]
/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: drtkdldjstm (drtkdldjstm-secudaim). Use `wandb login --relogin` to force relogin


Step,Training Loss
100,2.320600
200,1.081100
300,0.777200
400,0.542700
500,0.393100
600,0.291900
700,0.243500
800,0.214600
900,0.182200
1000,0.172700


/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/peft/tuners/lora/bnb.py:272: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


## Model Import

In [8]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID google/gemma-2-9b-it
    base_model = "google/gemma-2-9b-it"
    model_id = "finance_gemma2-9b-it_20240822_v2"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = Gemma2ForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        #temperature=0.2,
        return_full_text=False,
        max_new_tokens=256,
    )
    
    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

# LLM 파이프라인
llm = setup_llm_pipeline()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]
/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and sho

In [9]:
# pdf_databases = process_pdfs_from_dataframe(df, base_directory)

# # Save the pdf_databases dictionary to a file
# with open('pdf_databases.pkl', 'wb') as f:
#     pickle.dump(pdf_databases, f)
#     print("pdf_databases has been saved to pdf_databases.pkl")

# Later, load the pdf_databases dictionary from the file
with open('pdf_databases.pkl', 'rb') as f:
    pdf_databases = pickle.load(f)
    print("pdf_databases has been loaded from pdf_databases.pkl")

pdf_databases has been loaded from pdf_databases.pkl


In [ ]:
pdf_databases = process_pdfs_from_dataframe(test_df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]/lockard_ai/works/PEFT/venv_peft/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:05<00:41,  5.19s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:09<00:32,  4.64s/it]

Processing 보건복지부_노인장기요양보험 사업운영...


Processing PDFs:  33%|███▎      | 3/9 [00:14<00:27,  4.66s/it]

Processing 산업통상자원부_에너지바우처...


Processing PDFs:  44%|████▍     | 4/9 [00:18<00:22,  4.58s/it]

Processing 국토교통부_행복주택출자...


## Langchain을 이용한 추론

In [23]:
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Answering Questions"):
    # 소스 문자열 정규화
    print(row)
    source = normalize_string(row['Source'])
    question = row['Question']

Answering Questions: 100%|██████████| 98/98 [00:00<00:00, 1864.19it/s]

SAMPLE_ID                                     TEST_000
Source                           중소벤처기업부_혁신창업사업화자금(융자)
Source_path    ./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
Question               2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
Name: 0, dtype: object
SAMPLE_ID                                     TEST_001
Source                           중소벤처기업부_혁신창업사업화자금(융자)
Source_path    ./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
Question           중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
Name: 1, dtype: object
SAMPLE_ID                                           TEST_002
Source                                 중소벤처기업부_혁신창업사업화자금(융자)
Source_path          ./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
Question       중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
Name: 2, dtype: object
SAMPLE_ID                                     TEST_003
Source                           중소벤처기업부_혁신창업사업화자금(융자)
Source_path    ./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
Question               2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
Name: 3, dtype: object
SAMP

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

## Submission

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./QLoRA_gemma2-9b-it_v2_submission.csv", encoding='UTF-8-sig', index=False)


In [ ]:
submit_df

In [ ]:
df = pd.read_csv("./QLoRA_gemma2-9b-it_v2_submission.csv")
df